In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/en-core-web-md/en_core_web_md/meta.json
/kaggle/input/en-core-web-md/en_core_web_md/__init__.py
/kaggle/input/en-core-web-md/en_core_web_md/en_core_web_md-3.4.1/tokenizer
/kaggle/input/en-core-web-md/en_core_web_md/en_core_web_md-3.4.1/LICENSES_SOURCES
/kaggle/input/en-core-web-md/en_core_web_md/en_core_web_md-3.4.1/LICENSE
/kaggle/input/en-core-web-md/en_core_web_md/en_core_web_md-3.4.1/README.md
/kaggle/input/en-core-web-md/en_core_web_md/en_core_web_md-3.4.1/meta.json
/kaggle/input/en-core-web-md/en_core_web_md/en_core_web_md-3.4.1/accuracy.json
/kaggle/input/en-core-web-md/en_core_web_md/en_core_web_md-3.4.1/config.cfg
/kaggle/input/en-core-web-md/en_core_web_md/en_core_web_md-3.4.1/attribute_ruler/patterns
/kaggle/input/en-core-web-md/en_core_web_md/en_core_web_md-3.4.1/vocab/vectors
/kaggle/input/en-core-web-md/en_core_web_md/en_core_web_md-3.4.1/vocab/key2row
/kaggle/input/en-core-web-md/en_core_web_md/en_core_web_md-3.4.1/vocab/vectors.cfg
/kaggle/input/en-core-we

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split, GridSearchCV
import spacy

nlp = spacy.load("en_core_web_lg")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [3]:
summary_df = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/summaries_train.csv')
prompt_df = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/prompts_train.csv')
test_summary = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/summaries_test.csv')
test_prompts = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/prompts_test.csv')

In [4]:
from string import punctuation as punc
from collections import defaultdict
import spacy
from string import punctuation as punc
from tqdm import tqdm
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
import spacy
from textblob import TextBlob
import math
from collections import Counter
from nltk.util import ngrams

def extract_lemmas(text):
    doc = nlp(text)
    lemmas = []

    for token in doc:
        if not token.is_stop:
            lemmas.append(token.lemma_.lower())
    return ' '.join(lemmas)


def count_punc(text):
    return len([p for p in text if p in punc])

# Overlap Score

def tree_depth_from_token(token):
    """Calculate the depth of a subtree rooted at a token."""
    if not list(token.children):
        return 1
    else:
        return 1 + max(tree_depth_from_token(child) for child in token.children)

def average_dependency_tree_depth(doc):
    """Compute the average depth of the dependency tree."""
    root_tokens = [tok for tok in doc if tok.dep_ == 'ROOT']
    if not root_tokens:
        return 0
    depths = [tree_depth_from_token(token) for token in root_tokens]
    return sum(depths) / len(depths)

def process_text(doc):
    stops = 0
    num_nouns = 0
    num_verbs = 0
    num_adverbs = 0
    
    for token in doc:
        if token.is_stop:
            stops += 1
        else:
            if token.pos_ == "VERB":
                num_verbs += 1
            elif token.pos_ == "NOUN":
                num_nouns += 1
            elif token.pos_ == "ADV":
                num_adverbs += 1
                
    return stops, num_nouns, num_verbs, num_adverbs

def extract_features(row):
    doc = nlp(row['text'])
    context = nlp(row['prompt_text'])
    capital_error = sum(1 for sent in doc.sents if not sent.text[0].isupper())
    
    # 1. Length of the summary
    length = len(row['text'].split())
    
    # 2. Number of unique words
    unique_words = len(set([token.lemma_.lower() for token in doc]))
    
    #3. counter
    tokens = [token.lemma_.lower() for token in doc]
    total_tokens = len(tokens)
    word_counts = Counter(tokens)
    once = sum(1 for word, count in word_counts.items() if count == 1)
    twice = sum(1 for word, count in word_counts.items() if count == 2)
    
    # 4. Named entities
    num_entities = len(doc.ents)
    context_num_entities = len(context.ents)
    
    # 5. Average word length
    avg_word_len = round(sum([len(token.text) for token in doc])/length, ndigits=4)
    context_avg_word_len = round(sum([len(token.text) for token in context])/length, ndigits=4)
    
    # 6. Summary Polarity
    sentiment_polarity = TextBlob(row['text']).sentiment.polarity
    context_polarity = TextBlob(row['prompt_text']).sentiment.polarity
    error = sum(1 for token in doc if token.text.lower() not in english)
    
    # 7. Stopwords verbs and nouns
    stops,num_nouns, num_verbs, num_adverbs = process_text(doc)
    context_stops,context_nouns, context_verbs, context_adverbs = process_text(context)
    
    # 8. Numerical entities
    num_numerical_entities = len([ent for ent in doc.ents if ent.label_ == "CARDINAL"])
    
    # 9. Sentece count
    sentence = len([sent for sent in doc.sents])
    context_sentence = len([sent for sent in context.sents])
    avg_sentence = float(sentence / total_tokens)
    avg_unique_sentence = float(sentence / unique_words)
    context_unique_words = len(set([token for token in context]))
    avg_context_sentence = float(context_sentence / context_unique_words)
    
    # 10 Tree Depth
    average_tree_depth = average_dependency_tree_depth(doc)
    prob_word = [count/total_tokens for word, count in word_counts.items()]
    results = {
        "nsubj": 0,
        "amod": 0,
        "advmod": 0,
        "xcomp": 0,
        "acomp": 0,
        "past": 0,
        "present": 0
    }
    
    for token in doc:
        if "VERB" in token.pos_:
            if "VBD" in token.tag_ or "VBN" in token.tag_:
                results["past"] += 1
            elif "VBG" in token.tag_ or "VBP" in token.tag_ or "VBZ" in token.tag_:
                results["present"] += 1
        # Check if the token's dependency is one of the desired dependencies
        if token.dep_ in results:
            results[token.dep_] += 1
            
    # Unigram similarity
    summary_set = set(token.text for token in doc)
    context_set = set(token.text for token in context)
    intersection = len(context_set.intersection(summary_set))
    difference = len(context_set.difference(summary_set))
    union = len(context_set.union(summary_set))
    precision = float(intersection / len(summary_set))
    recall = float(intersection / len(context_set))
    jaccard_similarity = float(intersection/union)
    overlap_score = intersection/float(min(len(context_set), len(summary_set)) + 1e-9)
    # Dice's Coefficient
    dice_coefficient = (2 * jaccard_similarity)/(1 + jaccard_similarity)
    f1_score = float(2 * ((precision * recall)/(precision + recall + 1e-8)))
    
    
    # Bigram Similarity
    answer_bigrams = set(list(ngrams(row['text'].lower().split(), 2)))
    context_bigrams = set(list(ngrams(row['prompt_text'].lower().split(), 2)))
    intersection_bigram = len(answer_bigrams.intersection(context_bigrams))
    union_bigram = len(answer_bigrams.union(context_bigrams))
    precision_bigram = intersection_bigram / (len(answer_bigrams) + 1e-9)
    recall_bigram = intersection_bigram / (len(context_bigrams)+ 1e-9)
    jaccard_bigram = intersection_bigram/(union_bigram + 1e-9)
    f1_bigram = 2 * ((precision_bigram * jaccard_bigram)/(precision_bigram + jaccard_bigram + 1e-8))
    
    # Organizing features in a dictionary
    features = {
        'length': length,
        'num_chars': len(row['text']),
        "capital_error": capital_error,
        "intersection_bigram": intersection_bigram,
        "union_bigram": union_bigram,
        "jaccard_bigram": jaccard_bigram,
        "precision_bigram": precision_bigram,
        "f1_bigram": f1_bigram,
        'sentence': sentence,
        "context_sentence": context_sentence,
        "avg_sentence": avg_sentence,
        "avg_context_sentence": avg_context_sentence,
        'avg_unique_sentence':avg_unique_sentence,
        "error": error,
        'unique_words': unique_words,
        'num_entities': num_entities,
        "context_num_entities": context_num_entities,
        'avg_word_len': avg_word_len,
        "context_avg_word_len": context_avg_word_len,
        "stops": stops,
        "num_nouns": num_nouns,
        "num_verbs": num_verbs,
        "num_adverbs": num_adverbs,
        "context_nouns": context_nouns,
        "num_numerical_entities": num_numerical_entities,
        "sentiment_polarity": sentiment_polarity,
        "context_polarity": context_polarity,
        "nsubj": results['nsubj'],
        "amod": results['amod'],
        "advmod": results['advmod'],
        "xcomp": results['xcomp'],
        "acomp": results['acomp'],
        "past": results["past"],
        "present": results["present"],
        "average_dependency_tree_depth": average_tree_depth,
        "TTR": unique_words / (total_tokens + 1e-8),
        "RTTR": unique_words / (math.sqrt(total_tokens) + 1e-8),
        "CTTR": unique_words / (math.sqrt(total_tokens / 2) + 1e-8),
        # MATTR would require a window-based approach and is thus more involved
        "Herdan's C": math.log(unique_words) / (math.log(total_tokens) + 1e-8),
        "Dugast's U": math.log(total_tokens)**2 / ((math.log(total_tokens) - math.log(unique_words)) + 1e-8),
        "Honoré's H": 100 * math.log(total_tokens) / (1 - once/(unique_words + 1e-8)),
        "Entropy": -sum(p * math.log(p) for p in prob_word),
        "Sichel’s S": twice,
        "Simpson’s D": sum((count/total_tokens)**2 for count in word_counts.values()),
        'intersection': intersection,
        'difference': difference,
        'union': union,
        'overlap_score': overlap_score,
        'precision': precision,
        'dice_coefficient': dice_coefficient,
        'jaccard_similarity': jaccard_similarity,
        'f1_score': f1_score
    }
    
    return features

def compute_similarity(row):
    source_vector = nlp(row['prompt_text']).vector
    answer_vector = nlp(row['text']).vector
    embedding_similarity = cosine_similarity([source_vector], [answer_vector])[0][0]
    euclidean = euclidean_distances([source_vector], [answer_vector])[0][0]
    pearson = np.corrcoef(source_vector.ravel(), answer_vector.ravel())[0, 1]
    
    return pd.Series([embedding_similarity, euclidean, pearson])

In [5]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import TruncatedSVD,PCA
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances

def preprocess_data(summary_df, prompt_df):
    train_df = summary_df.merge(right=prompt_df, how='inner', on='prompt_id')

    print('processing text embeddings......')
    train_df = train_df.assign(
      full_text='Title:\n' + train_df['prompt_title'] +
                '\nQuestion:\n' + train_df['prompt_question'] +
                '\nText:\n' + train_df['prompt_text']
      )
    
    # processing lemmas
    print('extracting features.....')
    data = pd.DataFrame(train_df.apply(extract_features, axis=1).tolist())
    
    print('processing features.....')
    # Process other columns
    train_df['summary_punctuation'] = train_df['text'].apply(count_punc)
    train_df['lemmas'] = train_df['text'].apply(extract_lemmas)
    train_df['full_text'] = train_df['full_text'].apply(extract_lemmas)

    cols = ['summary_punctuation',
            'content', 'wording']

    print('applying vectorizer.....')
    # Vectorization
    global vectorizer, pca
    vectorizer = CountVectorizer(ngram_range=(1,1))
    transformer = TfidfTransformer()
    transformed_df = transformer.fit_transform(vectorizer.fit_transform(train_df['full_text']))
    summary_transformed = transformer.fit_transform(vectorizer.transform(train_df['lemmas']))

    print('scaling features.....')
    # Scaling
    scaler = MinMaxScaler(feature_range=(0, 10))
    X_summary_scaled = scaler.fit_transform(transformed_df.toarray())
    summary_scaled = scaler.transform(summary_transformed.toarray())

    print('decomposing features.......')
    # PCA Decomposition
    components = round(transformed_df.shape[1] * (40/100))
    pca = TruncatedSVD(n_components=components,algorithm='arpack', random_state=42)
    pca_bag = pca.fit_transform(X_summary_scaled)
    summary_bag = pca.transform(summary_scaled)
    similarity_df = train_df.apply(compute_similarity, axis=1)
    similarity = similarity_df.values
    
    stacked_bag = np.hstack((pca_bag,summary_bag,similarity,data.values,train_df[cols].values))

    print('collecting outputs.........')
    student_ids = train_df['student_id'].values

    return stacked_bag, student_ids

In [6]:
from nltk.corpus import words
english = set(words.words())

In [7]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import time

start_time = time.time()

stacked_bag, student_ids = preprocess_data(summary_df,prompt_df)

end_time = time.time()
elapsed_time = end_time - start_time

print(f"Preprocessed function executed in: {elapsed_time:.2f} seconds")

processing text embeddings......
extracting features.....
processing features.....
applying vectorizer.....
scaling features.....
decomposing features.......
collecting outputs.........
Preprocessed function executed in: 2248.87 seconds


In [8]:
X_train,X_test,y_train,y_test = train_test_split(stacked_bag[:,:-2] ,stacked_bag[:,-2:],test_size=0.1,shuffle=True,random_state=11)

In [9]:
print('done')

done


In [10]:
def mcrmse(y_true, y_pred):
    return np.mean(np.sqrt(np.mean((y_true - y_pred)**2, axis=0)))

mcrmse_scorer = make_scorer(mcrmse, greater_is_better=False)

In [11]:
X_train.shape

(6448, 628)

In [12]:
from sklearn.model_selection import KFold
import lightgbm as lgb
start_time = time.time()

best_params = {
    'learning_rate': 0.01,
    'n_estimators': 600,
    'max_depth': 11,
    'subsample': 0.6,
    'colsample_bytree': 0.6,
#     'objective': 'reg:squarederror'
}

model_lgb = MultiOutputRegressor(lgb.LGBMRegressor(**best_params, random_state=64),n_jobs=-1)
score = []
kfold = KFold(n_splits=20, shuffle=True, random_state=64)
for train_idx, test_idx in tqdm(kfold.split(X_train, y_train), desc='training model'):
    X, y = X_train[train_idx], y_train[train_idx]
    test,test_y = X_train[test_idx], y_train[test_idx]
    model_lgb.fit(X, y)
    score.append(mcrmse(test_y,model_lgb.predict(test)))
    
end_time = time.time()
elapsed_time = end_time - start_time 
print(f"Preprocessed function executed in: {elapsed_time:.2f} seconds")
print(f'Mean Score: {np.mean(score):.2f}')

training model: 0it [00:00, ?it/s]/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_

Preprocessed function executed in: 930.69 seconds
Mean Score: 0.49


In [13]:
y_preds = model_lgb.predict(X_test)

In [14]:
mcrmse(y_test,y_preds)

0.47582979221924515

### Preprocessing for text data

In [15]:
def preprocess_data(summary_df, prompt_df):
    train_df = summary_df.merge(right=prompt_df, how='inner', on='prompt_id')

    print('processing text embeddings......')
    train_df = train_df.assign(
      full_text='Title:\n' + train_df['prompt_title'] +
                '\nQuestion:\n' + train_df['prompt_question'] +
                '\nText:\n' + train_df['prompt_text']
      )
    
    # processing lemmas
    print('extracting features.....')
    data = pd.DataFrame(train_df.apply(extract_features, axis=1).tolist())
    
    print('processing features.....')
    # Process other columns
    train_df['summary_punctuation'] = train_df['text'].apply(count_punc)
    train_df['lemmas'] = train_df['text'].apply(extract_lemmas)
    train_df['full_text'] = train_df['full_text'].apply(extract_lemmas)

    cols = ['summary_punctuation']

    print('applying vectorizer.....')
    # Vectorization
    transformer = TfidfTransformer()
    transformed_df = transformer.fit_transform(vectorizer.transform(train_df['full_text']))
    summary_transformed = transformer.fit_transform(vectorizer.transform(train_df['lemmas']))

    print('scaling features.....')
    # Scaling
    scaler = MinMaxScaler(feature_range=(0, 10))
    X_summary_scaled = scaler.fit_transform(transformed_df.toarray())
    summary_scaled = scaler.fit_transform(summary_transformed.toarray())

    print('decomposing features.......')
    # PCA Decomposition
    pca_bag = pca.transform(X_summary_scaled)
    summary_bag = pca.transform(summary_scaled)
    similarity_df = train_df.apply(compute_similarity, axis=1)
    similarity = similarity_df.values
    
    stacked_bag = np.hstack((pca_bag,summary_bag,similarity,data.values,train_df[cols].values))

    print('collecting outputs.........')
    student_ids = train_df['student_id'].values

    return stacked_bag, student_ids

In [16]:
test_df,ids = preprocess_data(test_summary, test_prompts)

processing text embeddings......
extracting features.....
processing features.....
applying vectorizer.....
scaling features.....
decomposing features.......
collecting outputs.........


In [17]:
y_preds = model_lgb.predict(test_df)

In [18]:
df_test = pd.DataFrame(y_preds,columns=['content','wording'],index=ids).reset_index()
df_test = df_test.rename({'index':'student_id'},axis=1)

In [19]:
def is_valid_float(x):
    return isinstance(x, float) and x == x  # This checks that x is not NaN since NaN != NaN in Python.

cols_to_check = ['wording', 'content']
df_test[cols_to_check] = df_test[cols_to_check].applymap(lambda x: x if is_valid_float(x) else 0.0)

df_test.to_csv("submission.csv", index=False)